# Convolutional Neural Network

In [1]:
# Image dimensions
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

### Importing the libraries

In [2]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt


In [3]:
tf.__version__

'2.9.1'

## Data Preprocessing

### Preprocessing the Training set

In [4]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

training_set = train_datagen.flow_from_directory('input/Training',
                                                 target_size = IMG_SIZE,
                                                 batch_size = BATCH_SIZE,
                                                 class_mode = 'categorical')
labels =training_set.class_indices
print(labels)


Found 5712 images belonging to 4 classes.
{'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}


### Preprocessing the Test set

In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('input/Testing',
                                            target_size = IMG_SIZE,
                                            batch_size = BATCH_SIZE,
                                            class_mode = 'categorical')

Found 1311 images belonging to 4 classes.


## Building the CNN

### Loading the pretrained DenseNet121 model

In [6]:
base_model = DenseNet121(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

29084464/29084464 [==============================] - 3s 0us/step


### Adding custom layers for multiclass classification

In [7]:

x = base_model.output
x = GlobalAveragePooling2D()(x)  # Convert feature maps to a vector
x = Dense(512, activation="relu")(x)  # Fully connected layer
x = Dense(256, activation="relu")(x)  # Additional fully connected layer
x = Dropout(0.2)(x) # Dropout layer
x = Dense(4, activation="softmax")(x)  # Output layer (4 classes)

### Creating a model

In [8]:
model = Model(inputs=base_model.input, outputs=x)

## Training the CNN

### Compiling the CNN

In [9]:
model.compile(optimizer = Adam(learning_rate=0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                              

### Training the CNN on the Training set and evaluating it on the Test set

In [10]:
model.fit(x = training_set, validation_data = test_set, epochs = 10)

Epoch 1/10
179/179 [==============================] - 763s 4s/step - loss: 0.6573 - accuracy: 0.7544 - val_loss: 0.4918 - val_accuracy: 0.8238
Epoch 2/10
179/179 [==============================] - 629s 4s/step - loss: 0.3940 - accuracy: 0.8580 - val_loss: 0.4529 - val_accuracy: 0.8452
Epoch 3/10
179/179 [==============================] - 540s 3s/step - loss: 0.3489 - accuracy: 0.8736 - val_loss: 0.3954 - val_accuracy: 0.8520
Epoch 4/10
179/179 [==============================] - 645s 4s/step - loss: 0.3101 - accuracy: 0.8862 - val_loss: 0.4149 - val_accuracy: 0.8368
Epoch 5/10
179/179 [==============================] - 712s 4s/step - loss: 0.2938 - accuracy: 0.8904 - val_loss: 0.3206 - val_accuracy: 0.8764
Epoch 6/10
179/179 [==============================] - 769s 4s/step - loss: 0.2710 - accuracy: 0.8997 - val_loss: 0.3015 - val_accuracy: 0.8863
Epoch 7/10
179/179 [==============================] - 595s 3s/step - loss: 0.2566 - accuracy: 0.9065 - val_loss: 0.3270 - val_accuracy: 0.8703

## Making a single prediction

In [11]:
import numpy as np
from keras import utils
test_image = utils.load_img('input/Testing/notumor/Te-no_0018.jpg', target_size = IMG_SIZE)
test_image = utils.img_to_array(test_image) /255.0
test_image = np.expand_dims(test_image, axis = 0) # Convert single image to a batch.
predicted_indice_arr = model.predict(test_image)
print(predicted_indice_arr)
prediction = np.argmax(predicted_indice_arr, axis=1)

print(labels)
final_prediction = [key for key, val in labels.items() if val == prediction[0]]
print(final_prediction)


1/1 [==============================] - 4s 4s/step
[[0.00457821 0.00131441 0.99219805 0.0019093 ]]
{'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}
['notumor']


In [13]:
test_image = utils.load_img('input/Testing/pituitary/Te-pi_0018.jpg', target_size = IMG_SIZE)
test_image = utils.img_to_array(test_image) / 255.0
test_image = np.expand_dims(test_image, axis = 0) # Convert single image to a batch.
predicted_indice_arr = model.predict(test_image)
print(predicted_indice_arr)
prediction = np.argmax(predicted_indice_arr, axis=1)

print(labels)
final_prediction = [key for key, val in labels.items() if val == prediction[0]]
print(final_prediction)


1/1 [==============================] - 0s 152ms/step
[[4.1122281e-04 6.3070487e-03 1.1145765e-02 9.8213595e-01]]
{'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}
['pituitary']


In [14]:
test_image = utils.load_img('input/Testing/glioma/Te-gl_0018.jpg', target_size = IMG_SIZE)
test_image = utils.img_to_array(test_image) / 255.0
test_image = np.expand_dims(test_image, axis = 0) # Convert single image to a batch.
predicted_indice_arr = model.predict(test_image)
print(predicted_indice_arr)
prediction = np.argmax(predicted_indice_arr, axis=1)

print(labels)
final_prediction = [key for key, val in labels.items() if val == prediction[0]]
print(final_prediction)


1/1 [==============================] - 0s 132ms/step
[[9.9912578e-01 2.8344989e-04 5.8181805e-04 8.9706318e-06]]
{'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}
['glioma']


In [15]:
test_image = utils.load_img('input/Testing/meningioma/Te-me_0045.jpg', target_size = IMG_SIZE)
test_image = utils.img_to_array(test_image) / 255.0
test_image = np.expand_dims(test_image, axis = 0) # Convert single image to a batch.
predicted_indice_arr = model.predict(test_image)
print(predicted_indice_arr)
prediction = np.argmax(predicted_indice_arr, axis=1)

print(labels)
final_prediction = [key for key, val in labels.items() if val == prediction[0]]
print(final_prediction)


1/1 [==============================] - 0s 119ms/step
[[0.0129457  0.80719244 0.04121073 0.13865109]]
{'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}
['meningioma']
